## Context and Motivation
### In this course, neural networks are not treated as black boxes but as architectural components whose design choices affect performance, scalability, and interpretability.
This assignment focuses on convolutional layers as a concrete example of how inductive bias is introduced into learning systems.

Rather than following a recipe, students will select, analyze, and experiment with a convolutional architecture using a real dataset.

## Learning Objectives

By completing this assignment, students will be able to:

- **Understand** the role and mathematical intuition behind convolutional layers.  
- **Analyze** how architectural decisions (kernel size, depth, stride, padding) affect learning.  
- **Compare** convolutional layers with fully connected layers for image-like data.  
- **Perform** a minimal but meaningful exploratory data analysis (EDA) for neural network tasks.  
- **Communicate** architectural and experimental decisions clearly.


## Dataset Selection

##  Dataset Constraints

The dataset used in this assignment must meet the following requirements:

- **Image-based**: Data should be represented as 2D or 3D tensors.  
- **At least 2 classes**: Ensure the dataset contains a minimum of two distinct categories.  
- **Memory-friendly**: The dataset must fit entirely in memory on a standard laptop or cloud notebook environment.


In [3]:
%pip install numpy matplotlib tensorflow tensorflow-datasets


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/5.3 MB ? eta -:--:--
   --- ------------------------------------ 0.5/5.3 MB 1.5 MB/s eta 0:00:04
   ----------- ---------------------------- 1.6/5.3 MB 3.5 MB/s eta 0:00:02
   --------------------------- ------------ 3.7/5.3 MB 5.5 MB/s eta 0:00:01
   ---------------------------------------- 5.3/5.3 MB 6.5 MB/s  0:00:00
   ---------------------------------------- 0.0/27.5 MB ? eta -:--:--
   -- ------------------------------------- 1.6/27.5 MB 8.4 MB/s eta 0:00:04
   ----- ---------------------------------- 3.9/27.5 MB 10.2 MB/s eta 0:00:03
   ------- -------------------------------- 5.5/27.5 MB 9.1 MB/s eta 0:00:0